In [6]:
# 지역코드와 기간을 이용하여 해당기간, 해당지역의 아파트 매매 신고자료를 제공하는 아파트 매매 신고 정보 조회
import requests
import pprint


url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
params ={'serviceKey' : 'IO/EdnZ45auz8e6mcFKmXH/ZDn7lirSzzkWblTjoMfS331/hBMrvQ5v3KLTs6yfo1gM4Kdp0kRJaYqcRbkvt5w==', 'LAWD_CD' : '11110', 'DEAL_YMD' : '201512' }

response = requests.get(url, params=params)
print(response.status_code)
# print(response.content)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

200
('<?xml version="1.0" encoding="UTF-8" '
 'standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL '
 'SERVICE.</resultMsg></header><body><items><item><거래금액>      '
 '82,500</거래금액><거래유형> </거래유형><건축년도>2008</건축년도><년>2015</년><등기일자> </등기일자><법정동> '
 '사직동</법정동><아파트>광화문스페이스본(101동~105동)</아파트><월>12</월><일>10</일><전용면적>94.51</전용면적><중개사소재지> '
 '</중개사소재지><지번>9</지번><지역코드>11110</지역코드><층>11</층><해제사유발생일> </해제사유발생일><해제여부> '
 '</해제여부></item><item><거래금액>      60,000</거래금액><거래유형> '
 '</거래유형><건축년도>1981</건축년도><년>2015</년><등기일자> </등기일자><법정동> '
 '당주동</법정동><아파트>롯데미도파광화문빌딩</아파트><월>12</월><일>22</일><전용면적>149.95</전용면적><중개사소재지> '
 '</중개사소재지><지번>145</지번><지역코드>11110</지역코드><층>8</층><해제사유발생일> </해제사유발생일><해제여부> '
 '</해제여부></item><item><거래금액>     130,000</거래금액><거래유형> '
 '</거래유형><건축년도>2004</건축년도><년>2015</년><등기일자> </등기일자><법정동> '
 '내수동</법정동><아파트>킹스매너</아파트><월>12</월><일>8</일><전용면적>194.43</전용면적><중개사소재지> '
 '</중개사소재지><지번>110-15</지번><지역코드>11110</지역코드><층>6</층><해제사유발생일> </해제사유발생일><해제여부> '
 '</해제여부></item><ite

In [7]:
### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

## 각 컬럼 값 ## (포털 문서에서 꼭 확인하세요)
"""
LAWD_CD: 각 지역별 코드
DEAL_YMD: 월 단위 신고자료
""" 

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')
print(rows)

[<item><거래금액>      82,500</거래금액><거래유형> </거래유형><건축년도>2008</건축년도><년>2015</년><등기일자> </등기일자><법정동> 사직동</법정동><아파트>광화문스페이스본(101동~105동)</아파트><월>12</월><일>10</일><전용면적>94.51</전용면적><중개사소재지> </중개사소재지><지번>9</지번><지역코드>11110</지역코드><층>11</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>      60,000</거래금액><거래유형> </거래유형><건축년도>1981</건축년도><년>2015</년><등기일자> </등기일자><법정동> 당주동</법정동><아파트>롯데미도파광화문빌딩</아파트><월>12</월><일>22</일><전용면적>149.95</전용면적><중개사소재지> </중개사소재지><지번>145</지번><지역코드>11110</지역코드><층>8</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>     130,000</거래금액><거래유형> </거래유형><건축년도>2004</건축년도><년>2015</년><등기일자> </등기일자><법정동> 내수동</법정동><아파트>킹스매너</아파트><월>12</월><일>8</일><전용면적>194.43</전용면적><중개사소재지> </중개사소재지><지번>110-15</지번><지역코드>11110</지역코드><층>6</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>     105,000</거래금액><거래유형> </거래유형><건축년도>2004</건축년도><년>2015</년><등기일자> </등기일자><법정동> 내수동</법정동><아파트>경희궁의아침2단지</아파트><월>12</월><일>14</일><전용면적>124.17</전용면적><중개사소재지> </중개사소재지><지번>71</지번><지역코드>11110</지역코드><층>8</층><해제사유

In [8]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [10]:
#xml값 DataFrame으로 만들기
df = pd.DataFrame(row_list, columns=name_list)


In [11]:
df.head()

,거래금액,거래유형,건축년도,년,등기일자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"82,500",,2008,2015,,사직동,광화문스페이스본(101동~105동),12,10,94.51,,9,11110,11,,
1,"60,000",,1981,2015,,당주동,롯데미도파광화문빌딩,12,22,149.95,,145,11110,8,,
2,"130,000",,2004,2015,,내수동,킹스매너,12,8,194.43,,110-15,11110,6,,
3,"105,000",,2004,2015,,내수동,경희궁의아침2단지,12,14,124.17,,71,11110,8,,
4,"120,000",,2003,2015,,내수동,경희궁파크팰리스,12,24,146.33,,95,11110,4,,


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   거래금액     49 non-null     object
 1   거래유형     49 non-null     object
 2   건축년도     49 non-null     object
 3   년        49 non-null     object
 4   등기일자     49 non-null     object
 5   법정동      49 non-null     object
 6   아파트      49 non-null     object
 7   월        49 non-null     object
 8   일        49 non-null     object
 9   전용면적     49 non-null     object
 10  중개사소재지   49 non-null     object
 11  지번       49 non-null     object
 12  지역코드     49 non-null     object
 13  층        49 non-null     object
 14  해제사유발생일  49 non-null     object
 15  해제여부     49 non-null     object
dtypes: object(16)
memory usage: 6.2+ KB
